# `CLOUD.jl` - 2D linear advection example

In [99]:
using CLOUD, OrdinaryDiffEq, TimerOutputs

Define the physical problem

In [100]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [101]:
M = 4
p = 8
p_map = 4
form = WeakConservationForm()
strategy = ReferenceOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [102]:
reference_approximation = ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGLQuadrature(),
    facet_quadrature_rule=LGLQuadrature(),
    mapping_degree=p_map, N_plot=50,
    operator_algorithm=DefaultOperatorAlgorithm())

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [103]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

reset_timer!(timer)
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
print_timer(timer)

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            1.66s /  99.1%           2.33GiB /  99.9%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 rhs!                   4.81k    1.65s  100.0%   342μs   2.33GiB  100.0%   507KiB
   apply operators      77.0k    996ms   60.5%  12.9μs   1.16GiB   49.7%  15.7KiB
     mass matrix solve  77.0k    635ms   38.5%  8.24μs    108MiB    4.5%  1.44KiB
     volume terms       77.0k    261ms   15.8%  3.39μs    886MiB   37.2%  11.8KiB
     facet terms        77.0k   41.6ms    2.5%   541ns   81.0MiB    3.4%  1.08KiB
   gather ext state     77.0k    225ms   13.6%  2.92μs    459MiB   19.3%  6.11KiB
   eval num flu

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [104]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
4.2834865466010304e-5
Conservation (initial/final/diff):
[2.9456931685545307e-11][2.945518308428152e-11][-1.7486012637846216e-15]
Energy (initial/final/diff):
[0.12499999985776973][0.12499996352315211][-3.633461762020396e-8]


Write output to VTK file

In [105]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));